In [132]:
# Import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import time
import pandas as pd

In [133]:
# Setup browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Get the webpage for scraping articles and pass to Soup
browser.visit("https://mars.nasa.gov/news")
time.sleep(2)
soup = BeautifulSoup(browser.html, 'html.parser')

In [134]:
# Get the first article on the page
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

In [135]:
# Visit the next site for images
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
time.sleep(2)
browser.links.find_by_partial_text('FULL').click()
browser.links.find_by_partial_text('more info').click()

In [136]:
# Get the featured image
soup = BeautifulSoup(browser.html, 'html.parser')
featured_image_url = "https:" + soup.find_all('div', class_='download_tiff')[1].a['href']

In [139]:
# Visit the next site for Mars facts
browser.visit("https://space-facts.com/mars/")
time.sleep(2)
soup = BeautifulSoup(browser.html, 'html.parser')
table = soup.find('table', id="tablepress-p-mars-no-2").find_all('tr')

mars_info = []
for row in table:
    info = row.text.split(':')
    mars_info.append({
        "Attribute":info[0],
        "Info":info[1]
    })
table_html = pd.DataFrame(mars_info).to_html()

,Attribute,Info
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [160]:
# Visit the next site for hemispheres and grab all the links
browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
time.sleep(1)
soup = BeautifulSoup(browser.html,'html.parser')
links = soup.find_all('a', class_="itemLink product-item")

# Visit all of the links, collecting images along the way (every other link is an image, so skip it)
images = []
for page in links[1::2]:
    browser.visit("https://astrogeology.usgs.gov/" + page["href"])
    time.sleep(2)
    soup = BeautifulSoup(browser.html,'html.parser')

    images.append({
        "title":page.h3.text,
        "url":soup.find('a', text='Original')["href"]
    })


[{'title': 'Cerberus Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]